# Détectez des faux billets avec un algorithme Python

Ce notebook contient l'algorithme retenu pour la détection des faux billets à savoir la régression logistique avec GridSearchCV. Le travail de préparation et de sélection du modèle est présenté dans le fichier detection-faux-billets-oncfm-preparation-donnees.ipynb.

**Sommaire**

1. [Importation du modèle](#paragraph1)
2. [ Choix du fichier à analyser ](#paragraph2)    
3. [ Vérification des données et détection](#paragraph3)
4. [ Affichage des résultats](#paragraph3)


**Importation du modèle :**


In [44]:
import pandas as pd
import numpy as np
import joblib
from pathlib import Path  # importer la classe Path du module pathlib.

# Définit le modèle à importer et son chemin
model_folder = Path.cwd() / 'model'
model_file = 'logistic-regression-gs.joblib'
model_path = Path(model_folder / model_file)

# Charge le modèle si le fichier 'joblib' est présent au bon emplacement
if not model_folder.is_dir():
    raise FileNotFoundError(
        f'Impossible de charger le modèle : le dossier \'{model_folder}\''
        f' n\'existe pas.'
    )
elif not model_path.is_file():
    raise FileNotFoundError(
        f'Impossible de charger le modèle : le fichier \'{model_file}\''
        f' est manquant.'
    )
else:
    clf = joblib.load(model_path)
    print('Chargement du modèle effectué avec succès.')


Chargement du modèle effectué avec succès.


**Choix du fichier à analyser :**


In [45]:
# Initialise le chemin du dossier contenant les fichiers à tester
data_folder = Path.cwd() / 'datasets'

# Création d'une variable J qui pourrait être utilisée plus tard pour stocker l'indice du fichier sélectionné 
#par l'utilisateur parmi la liste des fichiers CSV disponibles.
# Initialise l'indice du fichier sélectionné : 
j=-1   # j=-1 aucun fichier n'a encore été sélectionné.

# Vérifie la présence du dossier 'datasets', liste les fichiers CSV qu'il
# contient et récupère le choix utilisateur
if not data_folder.is_dir():
    raise FileNotFoundError(f'Le dossier {data_folder} n\'existe pas.')
else:
    csv_files = [x for x in list(data_folder.glob('*.csv')) if x.is_file()]  # Créer une liste de tous les fichiers qui ont l'extension .csv 
    if len(csv_files) != 0:       # Vérification de la présence de fichiers CSV
        print('Fichiers disponibles pour le test :')
        for i, file in enumerate(csv_files):
            print(f'{i} | {file.parts[-1]}')
    else:
        raise FileNotFoundError(
            'Il n\'y a pas de fichiers CSV à tester dans votre dossier.')

Fichiers disponibles pour le test :
0 | billets.csv


In [46]:
while j not in range(0, len(csv_files)):
    j = int(input('Indiquez le numéro du fichier à tester.'))

print(f'\nVous avez choisi le fichier \'{csv_files[j].parts[-1]}\'.')

Indiquez le numéro du fichier à tester.0

Vous avez choisi le fichier 'billets.csv'.


**Vérification des données et détection :**

In [52]:
# Initialise le set des colonnes que doit contenir le fichier à tester
features = {'diagonal', 'height_left', 'height_right', 'margin_low',
            'margin_up', 'length', 'id'}

# Importe le fichier CSV dans un DataFrame
df = pd.read_csv(csv_files[j])

# Crée un set avec le nom des colonnes du DataFrame
df_features = set(df.columns)

# Vérifie que les features nécessaires sont présentes
if len(features - df_features) != 0:
    raise Exception(
        f'Impossible de tester le fichier, les variables suivantes sont '
        f'manquantes : {list(features - df_features)}')

# Vérifie qu'il n'y a pas de valeurs manquantes.
elif df.isna().any().any():
    raise Exception(
        f'Impossible de tester le fichier, les variables suivantes ont '
        f'des valeurs manquantes : {df.columns[df.isna().any()].to_list()}')

# Effectue la prédiction
else:
    ids = df['id']
    X = df.drop(columns='id')
    y_pred = clf.predict(X)

**Affichage des résultats :**


In [53]:
# Crée un DataFrame contenant les résultats de la prédiction
df_results = pd.DataFrame(
    zip(ids, y_pred, clf.predict_proba(X)[:, 0], clf.predict_proba(X)[:, 1]),
    columns=['id', 'is_genuine', 'prob_0', 'prob_1']
)

# Si y_pred contient des valeurs booléennes, pas besoin de mapper
# Si y_pred contient 0/1, mappez-les
if y_pred.dtype != bool:
    df_results['is_genuine'] = df_results['is_genuine'].map({0: False, 1: True})
# Calcul des estimations de probabilité
df_results['probability_estimates'] = np.where(
    df_results['is_genuine'], df_results['prob_1'], df_results['prob_0'])

# Supprimez les colonnes inutiles
df_results = df_results.drop(columns=['prob_0', 'prob_1'])


# Affiche les résultats de la détection
fake_banknote_list = list(df_results.loc[df_results['is_genuine'] == False, 'id'])

if len(fake_banknote_list) == 0:
    print('Aucun faux billet n\'a été détecté.')
else:
    print(f'Id des faux billets détectés : {fake_banknote_list}\n')
    print('Détails de la détection :\n')
    print(df_results)


Id des faux billets détectés : ['A_1', 'A_2', 'A_3']

Détails de la détection :

    id  is_genuine  probability_estimates
0  A_1       False               0.999734
1  A_2       False               0.999966
2  A_3       False               0.999961
3  A_4        True               0.968823
4  A_5        True               0.999984


In [51]:
# Crée un DataFrame contenant les résultats de la prédiction
df_results = pd.DataFrame(
    zip(ids, y_pred, clf.predict_proba(X)[:, 0], clf.predict_proba(X)[:, 1]),
    columns=['id', 'is_genuine', 'prob_0', 'prob_1']
)
# Renommer la colonne 'is_genuine' en 'billet_authentique'
df_results = df_results.rename(columns={'is_genuine': 'billet_authentique'})

# Calcul des estimations de probabilité
df_results['probability_estimates'] = np.where(
    df_results['billet_authentique'], df_results['prob_1'], df_results['prob_0'])

# Supprimez les colonnes inutiles
df_results = df_results.drop(columns=['prob_0', 'prob_1'])

# Affiche les résultats de la détection
fake_banknote_list = list(df_results.loc[df_results['billet_authentique'] == False, 'id'])

if len(fake_banknote_list) == 0:
    print('Aucun faux billet n\'a été détecté.')
else:
    print(f'Id des faux billets détectés : {fake_banknote_list}\n')
    print('Détails de la détection :\n')
    print(df_results)


Id des faux billets détectés : ['A_1', 'A_2', 'A_3']

Détails de la détection :

    id  billet_authentique  probability_estimates
0  A_1               False               0.999734
1  A_2               False               0.999966
2  A_3               False               0.999961
3  A_4                True               0.968823
4  A_5                True               0.999984
